# ML mini project #1

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [4]:
X_train = X_train_full / 255.
y_train = y_train_full.astype(float)

In [5]:
X_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [7]:
y_train

array([9., 0., 0., ..., 3., 0., 5.])

In [22]:
def build_model(): 
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    
    lrelu = lambda x: tf.keras.activations.relu(x, alpha=0.2)
    model.add(keras.layers.Dense(300, activation=lrelu))
    model.add(keras.layers.Dense(150, activation=lrelu))
    model.add(keras.layers.Dense(100, activation=lrelu))
    model.add(keras.layers.Dense(10, activation="softmax"))
    
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="sgd",
                  metrics=["accuracy"])
    return model

In [23]:
k = 10
num_val = len(X_train) // k
num_epochs = 50
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 984us/step - loss: 0.3608 - accuracy: 0.8827
processing fold # 1
188/188 [==============================] - 0s 1ms/step - loss: 0.3027 - accuracy: 0.8972
processing fold # 2
188/188 [==============================] - 0s 890us/step - loss: 0.3103 - accuracy: 0.8932
processing fold # 3
188/188 [==============================] - 0s 893us/step - loss: 0.3224 - accuracy: 0.8893
processing fold # 4
188/188 [==============================] - 0s 1000us/step - loss: 0.3261 - accuracy: 0.8902
processing fold # 5
188/188 [==============================] - 0s 1ms/step - loss: 0.3334 - accuracy: 0.8893
processing fold # 6
188/188 [==============================] - 0s 1ms/step - loss: 0.3254 - accuracy: 0.8935
processing fold # 7
188/188 [==============================] - 0s 1ms/step - loss: 0.3377 - accuracy: 0.8867
processing fold # 8
188/188 [==============================] - 1s 3ms/step - loss: 0.3452 - accuracy: 0.8838
processing

In [24]:
all_scores

[0.8826666474342346,
 0.8971666693687439,
 0.8931666612625122,
 0.8893333077430725,
 0.8901666402816772,
 0.8893333077430725,
 0.8934999704360962,
 0.8866666555404663,
 0.8838333487510681,
 0.8920000195503235]

In [25]:
np.mean(all_scores)

0.8897833228111267